In [4]:
from IPython.display import Image

- https://github.com/deepseek-ai/DeepSeek-Math

## basics

- Data Collection and Decontamination: **revelant** data at scale
    - we create the DeepSeekMath Corpus, a large-scale high-quality pre-training corpus comprising 120B math tokens.
    - iterative pipeline;
    - seed corpus
        - a small but high-quality collection of math-related dataset
        - OpenWebMath
    - train a FastText model: classifier
        - revelant or irrelevant
    - Discover math-related domains
        - After the first iteration of data collection, numerous mathematical web pages remain uncollected,  ...
        - 扩召回；
- SFT
    -  500 steps bs 256:
        -  2^9 * 2^8 = 2 ^ 17 = 131072, 1m
- GRPO
- experiments
    - coding 对 math 的影响；

In [2]:
Image(url='./imgs/ds-math-data.png', width=400)

## GRPO

- RL
    - Actor & Env: Actor ($\text{action}=\pi_\theta(\text{obs})$)
        - 1. Env -> Actor: (obs = q);
        - 2. Actor -> Env: (action = **o**utput);
        - 3. Env -> Actor: reward;
    - LLM is the policy: $o=LLM(q, \theta)$

### TRPO => PPO => GRPO

In [11]:
Image(url='./imgs/pg_wss.png', width=400)

- policy learning: policy $\pi$ is parameterized $\pi_\theta$
    - 与之相对的是 q learning：学习的是 q function，由 q function 导出 action；
- REINFORCE algorithm: if agent selects an action (given state) that results in good outcome, then it should take that action more offen in the future. 也即是说这些 actions 能带来高 rewards 的 actions 应该被加强（REINFORCE）
    - $\pi(\cdot)=\pi(\cdot|s_t)$ 是一个概率分布
    - $\Delta_\theta=\nabla_\theta(-\log \pi(\cdot))\cdot R$
        - R 指引着更新的幅度；
    - $\Delta_\theta=\nabla_\theta(-\log \pi(\cdot))\cdot A$
        - $A=(r-b)$：advantage
            - $b$: baseline, $V(s)$ (only state)
            - $r$: state & action,

- TRPO
    - Trust region approach
    - 不过分相信 pg 的 gradient，约束一个 trust region
    - $L=\ell + D_{kl}(\pi_\theta, \pi_{\theta_{old}})$
        - KL 散度作为硬约束/软约束的形式出现；
- PPO
    - https://huggingface.co/blog/deep-rl-ppo
    $$
    L^{CLIP}(\theta)=\mathbb E[\min(r(\theta)A(s,a), \text{clip}(r(\theta), 1-\epsilon, 1+\epsilon)A(s,a))]
    $$
    - 裁剪概率比（Probability Ratio）简化TRPO的约束，避免复杂的二阶优化。
    - an actor-critic RL algorithm
        - Actor: Policy model
        - Critic: Value Model
            - Critic 的价值估计与奖励模型的实际反馈结合，计算优势值（Advantage），衡量当前策略的改进空间。
    - surrogate objective ($\gt 0$, Clipped Surrogate Objective Function)
        - $r(\theta)=\frac{\pi_\theta(a|s)}{\pi_{\theta_{old}(a|s)}}$ (likelihood ratio, Importance Sampling Ratio)
        - PPO通过剪切概率比，限制其偏离范围（如 $[1-\epsilon, 1+\epsilon]$), 从而隐式约束策略更新幅度：
    - LLM + RL (PPO)
        - formula 部分是 token-wise (auto-regress step by step) 的；
- GRPO
    - GAE
        - 一般 Advantage Estimate（单步TD残差），使用单步时序差分（TD）误差估计优势：
            - $A(s_t,a_t)=r_t+\gamma V(s_{t+1})-V(s_t)$
        - GAE 通过多步TD残差的加权平均估计优势：
            - $A^{GAE(\lambda, \gamma)}(s_t,a_t)=\sum_{k=0}^\infty (\gamma\lambda)^k\delta_{t+k}$
                - $\delta_t=r_t+\gamma V(s_{t+1})-V(s_t)$
    - KL estimate
        - $r=\frac{\pi_{ref}}{\pi_\theta}$
        - $r-\log r-1$
            - which is guaranteed to be positive ??
            - taylor 展开后，$\log r \leq r-1$

In [5]:
Image(url='https://huggingface.co/blog/assets/93_deep_rl_ppo/recap.jpg', width=400)

- like situation 4, the advantage estimate is negative, we don't want to decrease further the probability of taking that action at that state. Therefore, the gradient is = 0 (since we're on a flat line), so we don't update our weights.
- ike in situation 5, the advantage is positive, we don't want to get too greedy. We already have a higher probability of taking that action at that state than the former policy. Therefore, the gradient is = 0 (since we're on a flat line), so we don't update our weights.

## towards to a unified paradigm

- https://karpathy.github.io/2016/05/31/rl/
    - https://www.youtube.com/watch?v=tqrcjHuNdmQ
    - sft & policy gradient
    - Deriving Policy Gradients.
- policy gradients 不只兼容了 RL 中的 policy-based methods，还兼容了 SFT（即 Supervised Learning）
    - supervised learning：$-\log p(\cdot|x)\rightarrow -\nabla \log p(\cdot|x)$
- Advantage: scaler that how much you want to encourage or discourage the action that you happen to take.
    - $A_i$ could be 1.0 if we eventually won in the episode that contained $x_i$ and -1.0 if we lost. 

In [7]:
Image(url='./imgs/sl_rollouts.png', width=400)

In [9]:
# positive advantage will make that action more likely in the future, for that state
# negative advantage will make that action less likely in the future, for that state
Image(url='./imgs/sl_vs_rl.png', width=400)